In [1]:
import pandas as pd
import bs4
import requests
import json
from io import StringIO


In [3]:
# single car example

url = 'https://evkx.net/models/tesla/model_y/model_y_long_range/chargingcurve/'

tables = pd.read_html(url)

CHARGE_TIMES_INDEX = 1

charge_time_table = tables[CHARGE_TIMES_INDEX]

In [4]:
model_y_charge_str_times = charge_time_table['Time'].tolist()



In [2]:
def hms_to_seconds(hms):

    hours = 0

    split_hours = hms.split('h')
    hours_index = 0
    m_s_index = 1


    # handle case where theres no hours
    if 'h' in hms:
        hours = int(split_hours[hours_index])
    else:
        m_s_index = 0
    
    split_minutes = split_hours[m_s_index].split('m')
    minutes_index = 0
    seconds_index = 1

    minutes = int(split_minutes[minutes_index])

    split_seconds = split_minutes[seconds_index].split('s')
    seconds_index = 0

    seconds = int(split_seconds[seconds_index])

    total_seconds = 60 * 60 * hours + 60 * minutes + seconds

    return total_seconds

hms_to_seconds('1 m 1 s')

61

In [3]:
def get_charge_times(site_text):
    tables = pd.read_html(StringIO(site_text), encoding='utf-8')
    
    CHARGE_TIMES_INDEX = 1
    charge_time_table = tables[CHARGE_TIMES_INDEX]

    hms_string_times = charge_time_table['Time'].tolist()

    return [hms_to_seconds(i) for i in hms_string_times]

In [6]:
model_y_charge_seconds = [hms_to_seconds(i) for i in model_y_charge_str_times]

In [11]:
makers_url = 'https://evkx.net/models/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Referer': 'https://www.example.com',
}

res = requests.get(makers_url, headers=headers)
res.raise_for_status()
makes_soup = bs4.BeautifulSoup(res.text, 'html.parser')

# get the car maker cards
maker_cards = makes_soup.select('div.container.p-3.mb-4.bg-body-tertiary.rounded.border')

cars = {}
for card in maker_cards[0:]:
    # extract the model name
    maker_name = card.select('h2')[0].get_text()
    print(maker_name)
    # extract the link
    link = card.select('a')[0].get('href')

    # follow link to get the models
    models_url = makers_url + link
    models_resp = requests.get(models_url, headers=headers)
    models_resp.raise_for_status()
    models_soup = bs4.BeautifulSoup(models_resp.text, 'html.parser')

    model_cards = models_soup.select('div.container.p-3.mb-4.bg-body-tertiary.rounded.border')

    models = {}
    for model in model_cards:
        model_name = model.select('h3')[0].get_text()
        link = model.select('a')[0].get('href')
        
        sub_model_url = models_url + link
        sub_models_resp = requests.get(sub_model_url, headers=headers)
        sub_models_resp.raise_for_status()
        sub_models_soup = bs4.BeautifulSoup(sub_models_resp.text, 'html.parser')

        sub_model_cards = sub_models_soup.select('div.container.p-3.mb-4.bg-body-tertiary.rounded.border')
        sub_models = {}

        for sub_model in sub_model_cards:
            sub_model_name = sub_model.select('h3')[0].get_text()
            charge_curve_selector = 'li:nth-child(3) > a:nth-child(1)'
            link = sub_model.select(charge_curve_selector)[0].get('href')
            charge_curve_url = sub_model_url + "/" + link
            charge_curve_resp = requests.get(charge_curve_url, headers=headers)
            charge_times = get_charge_times(charge_curve_resp.text)
            sub_models[sub_model_name] = charge_times
            print(sub_model_name)

        models[model_name] = sub_models
    cars[maker_name] = models
    
print("done")

Acura
Acura ZDX Type S
Aion
Aion Hyper GT 710 Supercharged
Audi
Audi e-tron 50
Audi e-tron 50 Sportback
Audi e-tron 55
Audi e-tron 55 Sportback
Audi e-tron S
Audi e-tron S Sportback
Audi e-tron GT
Audi RS e-tron GT
Audi Q4 45 e-tron quattro
Audi Q4 45 e-tron
Audi Q4 55 e-tron quattro
Audi Q4 Sportback 45 e-tron quattro
Audi Q4 Sportback 45 e-tron
Audi Q4 Sportback 55 e-tron quattro
Audi Q4 35 e-tron
Audi Q4 40 e-tron
Audi Q4 45 e-tron quattro (GEN1)
Audi Q4 50 e-tron quattro
Audi Q4 Sportback 35 e-tron
Audi Q4 Sportback 40 e-tron
Audi Q4 Sportback 45 e-tron quattro (GEN1)
Audi Q4 Sportback 50 e-tron quattro
Audi Q8 50 e-tron
Audi Q8 Sportback 50 e-tron
Audi Q8 55 e-tron
Audi Q8 Sportback 55 e-tron
Audi Q8 e-tron edition Dakar
Audi SQ8 e-tron
Audi SQ8 Sportback e-tron
BMW
BMW i4 eDrive35
BMW i4 eDrive40
BMW i4 M50
BMW i5 eDrive40
BMW i5 eDrive40 Touring
BMW i5 M60 xDrive
BMW i5 M60 xDrive Touring
BMW i5 xDrive40
BMW i7 eDrive50
BMW i7 M70 xDrive
BMW i7 xDrive60
BMW iX xDrive40
BMW iX xD

In [47]:
with open('charge_times.json', 'w') as f:
    json.dump(cars, f)
f.close()